# GEOPHYS 257 (Winter 2023)
[//]: <> (Notebook Author: Thomas Cullison, Stanford University, Jan. 2023)

## Multicore Parallelization of Matrix Multiplication Using Numba

In this lab we will be using Numba to accelerate matrix-matrix multiplications by exploiting parallelism. Even most laptops today have Multicore CPUs, where a *core* is a microprocessor, and each core is usually a copy of the each other core. Accelerating the marrix-matrix multiplication operation is a good analog to accelerating other types of operators and computationally intense kernels, codes, and algorithms. Furthermore, the structure of matricies makes matrix-matrix multiplication a good place start learning how to parallelize code.


## External Resources
If you have any question regarding some specific Python functionality you can consult the official [Python documenation](http://docs.python.org/3/).

* [Numba](https://numba.readthedocs.io/en/stable/index.html): Documentation
* [Numba in 30 min](https://youtu.be/DPyPmoeUdcE): Conference presentation video


## Required Preperation
Please watch the following videos before starting the lab (each is pretty short):
* [Introduction to Parallel Computing](https://youtu.be/RNVIcm8-6RE)
* [Amdahl's Law](https://youtu.be/Axx2xuB-Xuo)
* [CPU Caching](https://youtu.be/KmairurdiaY)
* [Pipelining](https://youtu.be/zPmfprtdzCE)
* [Instruction Level Parallelism](https://youtu.be/ZoDUI3gTkDI)
* [Introduction to SIMD](https://youtu.be/o_n4AKwdfiA)


### Exercise 0

Please run the following cells.  Examine the result of the example function that makes use of the *timer* wrapper. Also please use this timer wrapper (defined below) for all the exercises that follow.

#### Load python modules (Note: you may need to install some Python packages for the modules below, e.g. Numba)

In [1]:
import numba
import numpy as np


from functools import wraps
from time import time, sleep
from itertools import permutations

#### Define a function-decorator for timing the runtime of your functions

Below is some code that you can use to wrap your functions so that you can time them individually.  The function defined imeadiately below the *timer()* is an example of how to use the warpper. In the cell that follows, execute this example function. (Note, for this timer, were are making use of something called *decorators*, but a discussion about this feature is outside the scope of this class.)


In [2]:
# defining a function decorator for timing other functions
def mytimer(func):
    @wraps(func)
    def wrap(*args, **kwargs):
        t_start = time() # Students look here
        result = func(*args, **kwargs)
        t_end = time() # Students also look here. This is how you can time things inside functions/classes
        print(f'Function: \'{func.__name__}({kwargs})\' executed in {(t_end-t_start):4.3f}s\n')
        return result
    return wrap
    

# Example of how to use. NOTE the "@mytimer" stated just above the function definition
@mytimer
def example_sum_timer_wrap(N):
    """ Sum the squares of the intergers, i, contained in [1-N] """
    return np.sum(np.arange(1,N+1)**2)

#### Run the example function

Run the example function for each of the following instances of $N = 10^5, 10^6, 10^7, 10^8$. Please examine the results, in particular, how the runtime changes with respect to $N$.


**Answer** the following questions in the markdown cell that follow the code. 
* For each factor-of-ten increase in $N$, roughly how much longer was the runtime of the function?
* Does this slowdown in the runtime make sense? Why or why not?

In [10]:
# Call the example function above for each value of N (try making one-call first, then loop)
example_sum_timer_wrap(10**5)
example_sum_timer_wrap(10**6)
example_sum_timer_wrap(10**7)
example_sum_timer_wrap(10**8)

Function: 'example_sum_timer_wrap({})' executed in 0.001s

Function: 'example_sum_timer_wrap({})' executed in 0.022s

Function: 'example_sum_timer_wrap({})' executed in 0.212s

Function: 'example_sum_timer_wrap({})' executed in 1.026s



672921401752298880

#### Your answer for Exerciese 0 questions here:

For the first factor-of-ten increase in N the function took ~20x longer, for the second increase the function took ~10x longer, and for the third increase the function took ~5x longer. This runtime slowdown makes sense, as the larger N is, the more parallelizable the function is to Numba, so the increase in runtime gets smaller.


<br><br>
### Exercise 1: Naive matrix multiplication: 

For this exercise, we will write our own matrx-matrix multiplication function.  We are goint to be naieve about it, so we want to loop over individual indices, instead of using slicing (in the next section, we will parallelize this code and we want to see how well Numba does at speeding up the naive code).

#### Tasks for this exercise

1. Write a function with that calculates matrix-matrix multiplication such that $C = A\cdot B$, where $A$, $B$, and $C$ are 2D numpy arrays. Make the dtype for the $C$ matrix the same as the dtype for $A$. Below is a stencil you can start with. test your results for accuracy and performance against the np.dot() function. To time the np.dot() function, you can wrap it in another function and use the *mytimer* wrapper; however, please opy the code for testing the A and B dimensions into your function wrapper for the np.dot() function.  This keeps the runtime comparison as a more "apples-to-apples" like comparison.

```python
@mytimer
def my_naive_matmul(A:np.ndarray, B:np.ndarray):
    """ This is a naive matrix-multiplication function """ 
    
    # 1. 
    # check that A and B have appropriate dimensions for multiplication
    
    # 2. 
    # construnct a 2D numpy array for matrix C, that is filled with zerros 
    # and that has the appropriate dimensions such taht C=A*B is a valid 
    # equation and operation
    
    # 3
    # Write three nested for loops, with indices, i,j,k to solve the matrix-multiplication
    # e.g.
    # for i in range(<val>):
    #   for j in range(<val>):
    #     for k in range(<val>):
    #       C[<c_row_index>,<c_col_index>] += A[<a_row_index>,<a_col_index>]*B[<b_row_index>,<b_col_index>]
    
    # 4
    # return the 2D numpy array for C
    return C
```
<br>

2. Define your functions in the cell below.
3. Test and compare the accuracy and runtime of your functions in the next cell.
    - Test with non-square Matrices: $A \in \mathbb{R}^{N\times K}$ and $B \in \mathbb{R}^{K \times M}$. With $N = 64$, $K = 32$, and $M = 128$.
    - Test with a square Matrices: $A,B \in \mathbb{R}^{N\times N}$. For the cases when $N = 64, 128,$ and $256$.
    - Test the following three cases:
        - Case-1. $A$ and $B$ **both** have dtype=np.**float32** (make sure that $C$ is also of dtype=np.**float32**)
        - Case-2. $A$ has dtype=np.**float64**, but $B$ has dtype=np.**float32**
        - Case-3. $A$ and $B$ **both** have dtype=np.**float64** (make sure that $C$ is also of dtype=np.**float64**)
    - For all three case above:
        - Calculate and show the error by computing the sum of the differnece between the $C$ matrices computed from numpy.dot() and your my_naive_matmul() functions. Assume that numpy.dot() is correct
        - Calculate and show the *speedup* that the faster function has versus the slower function.
        - Comment on the which of the three cases is fastest, and comment on what the speedup of the fastest case is and why it is the fastest case.
4. Create your matrices using random numbers. An example is shown below (feel free to copy this).

```python
A = np.random.rand(N, K)
B = np.random.rand("for-you-to-figure-out")
```

<br>

#### Write function definitions for Exercise 1 below

In [3]:
# Define your functions for Exercise 1 here.

@mytimer
def my_naive_matmul(A:np.ndarray, B:np.ndarray):
    """ This is a naive matrix-multiplication function """ 
    
    # 1. 
    # check that A and B have appropriate dimensions for multiplication
    if (np.shape(A)[1] != np.shape(B)[0]):
        return
    
    # 2. 
    # construnct a 2D numpy array for matrix C, that is filled with zerros 
    # and that has the appropriate dimensions such taht C=A*B is a valid 
    # equation and operation
    C = np.zeros((np.shape(A)[0],np.shape(B)[1]))
    
    # 3
    # Write three nested for loops, with indices, i,j,k to solve the matrix-multiplication
    # e.g.
    # for i in range(<val>):
    #   for j in range(<val>):
    #     for k in range(<val>):
    #       C[<c_row_index>,<c_col_index>] += A[<a_row_index>,<a_col_index>]*B[<b_row_index>,<b_col_index>]
    for i in range(np.shape(C)[0]):
        for j in range(np.shape(C)[1]):
            for k in range(np.shape(A)[1]):
                C[i,j] += A[i,k]*B[k,j]
    
    # 4
    # return the 2D numpy array 
    return C

@mytimer
def timed_npdot(A:np.ndarray, B:np.ndarray):
    return np.dot(A,B)


<br>

#### Write the test codes for Exercise 1 in the cell below.

In [16]:
# Test your code here for runtime and accuracy

# Non-square matrix test
print("Non-square matrices:")
A1 = np.random.rand(64,32)
B1 = np.random.rand(32,128)
C1naive = my_naive_matmul(A1,B1)
C1np = timed_npdot(A1,B1)
err1 = np.sum(abs(C1naive-C1np))
print("Error = ",err1)

# Square matrix test
print("\nSquare matrices, N=64:")
A2 = np.random.rand(64,64)
B2 = np.random.rand(64,64)
C2naive = my_naive_matmul(A2,B2)
C2np = timed_npdot(A2,B2)
err2 = np.sum(abs(C2naive-C2np))
print("Error = ",err2)

print("\nSquare matrices, N=128:")
A3 = np.random.rand(128,128)
B3 = np.random.rand(128,128)
C3naive = my_naive_matmul(A3,B3)
C3np = timed_npdot(A3,B3)
err3 = np.sum(abs(C3naive-C3np))
print("Error = ",err3)

print("\nSquare matrices, N=256:")
A4 = np.random.rand(256,256)
B4 = np.random.rand(256,256)
C4naive = my_naive_matmul(A4,B4)
C4np = timed_npdot(A4,B4)
err4 = np.sum(abs(C4naive-C4np))
print("Error = ",err4)

print("\nCase 1:")
A5 = np.float32(A4)
B5 = np.float32(B4)
C5naive = my_naive_matmul(A5,B5)
C5np = timed_npdot(A5,B5)
err5 = np.sum(abs(C5naive-C5np))
print("Error = ",err5)
print("Speedup ~ 10500")

print("\nCase 2:")
A6 = np.float64(A4)
B6 = np.float64(B4)
C6naive = my_naive_matmul(A6,B5)
C6np = timed_npdot(A6,B5)
err6 = np.sum(abs(C6naive-C6np))
print("Error = ",err6)
print("Speedup ~ 10700")

print("\nCase 3:")
C7naive = my_naive_matmul(A6,B6)
C7np = timed_npdot(A6,B6)
err7 = np.sum(abs(C7naive-C7np))
print("Error = ",err7)
print("Speedup ~ 9900")
print("This is consistently the fastest case. Case 2 is slower because the float32 array is changed to float64, and case 1 has much lower accuracy")

Non-square matrices:
Function: 'my_naive_matmul({})' executed in 0.293s

Function: 'timed_npdot({})' executed in 0.000s

Error =  3.5442759838133497e-12

Square matrices, N=64:
Function: 'my_naive_matmul({})' executed in 0.181s

Function: 'timed_npdot({})' executed in 0.000s

Error =  3.4923175462608924e-12

Square matrices, N=128:
Function: 'my_naive_matmul({})' executed in 1.478s

Function: 'timed_npdot({})' executed in 0.000s

Error =  2.8215652037033578e-11

Square matrices, N=256:
Function: 'my_naive_matmul({})' executed in 12.077s

Function: 'timed_npdot({})' executed in 0.001s

Error =  1.4549002003150235e-09

Case 1:
Function: 'my_naive_matmul({})' executed in 11.584s

Function: 'timed_npdot({})' executed in 0.001s

Error =  0.7439966191374907
Speedup ~ 10500

Case 2:
Function: 'my_naive_matmul({})' executed in 14.174s

Function: 'timed_npdot({})' executed in 0.002s

Error =  1.4547794080499443e-09
Speedup ~ 10700

Case 3:
Function: 'my_naive_matmul({})' executed in 9.823s

Fun

<br><br>
### Exercise 2: Using Numba to speed up matrix multiplication: 

For this exercise, numba to speedup our matrx-matrix multiplication function below. However, there is an interesting twist to this exercise. We will write six versions of the naive function you wrote above. One function for each of the six possible permutations of three loops used to calculate the multplication (i.e. ijk, ikj, jik, etc.)

#### The tasks for this exercise:
1. Make six copies your code for the my_naive_matmul(), one copy for each of the possible permutations and define them in the cell bellow:
    - One of these function will have the same loop order as the my_naive_matmul() function. 
    - However name these functions: numba_mul_\<perm\>(), where \<perm\> should be replaced by the specific loop order of the function.
2. In the cell that follows your function definitions, test and compare the accuracy and runtime of your functions against the numpy.dot() function.
    - Test with a square Matrices: $A,B \in \mathbb{R}^{N\times N}$. For the cases when $N = 128, 256,$ and $512$.
    - For each matrix set their dtype as: dtype=np.float64
    - Calculate and show the error between your functions and the numpy.dot() function. (Same as in Exercise 1.)
    - Calculate and show the *speedup* that the fastest function has versus the all the other functions.
    - You should notice that one of your permutation functions is faster than the others. For this case show the following:
        - Calculate and show the *speedup* that the fastest permutation function has versus the my_naive_matmul().
        - Calculate and show the *speedup* that the fastest permutation function has when $A$, $B$, and $C$ are all of dtype=np.float64 vs all are of dtype=np.float32.
3. Create your matrices using random numbers. (Same as in Exercise 1.) 
4. For each function, you need to add a function decorator for numba. Numba will *JIT* the function (**J**ust **I**n **T**ime compilation). 
    - Use the flag to keep a "cache" of the compiled code (not to be confused with CPU cache). **Note**: to get accurate timings, you will need to run your tests **twice**, because during the first run, the code is compiled and this compile time will be included in your runtime. After the first run, the compiled binary will be stored, so consecutive runs will be faster.
    - Use the flag to use *fast math*
    - Use the flag to disable the Python Global Interpretor Lock (GIL).
    - The code below should get you started, but it is incomplete.
    
```python   
    
@mytimer
@numba.njit(<flagname_caching>=<flag>, <flagname_fast_math>=<flag>, <flagname_no_gil>=<flag>)
def numba_mul_<perm>(A:np.ndarray, B:np.ndarray):
    """ This is a Numba accelerated matrix-multiplication function """ 
    
    # 1. 
    # check that A and B have appropriate dimensions for multiplication
    
    # 2. 
    # construnct a 2D numpy array for matrix C, that is filled with zerros 
    # and that has the appropriate dimensions such taht C=A*B is a valid 
    # equation and operation
    
    # 3
    # Write three nested for loops, with indices, i,j,k to solve the matrix-multiplication
    # e.g.
    # for i in numba.prange(<val>): # !!!! LOOK HERE !!!!
    #   for j in range(<val>):
    #     for k in range(<val>):
    #       C[<c_row_index>,<c_col_index>] += A[<a_row_index>,<a_col_index>]*B[<b_row_index>,<b_col_index>]
    
    # 4
    # return the 2D numpy array for C
    return C
```
    
6. Discuss your results in the markdown cell that follows your codes include in your discussion remarks about the questions asked in the markdown cell.


<br>

#### Write function definitions for Exercise 2 below

In [ ]:
# Define your functions for Exercise 2 here.

@mytimer
@numba.njit(cache=True, fastmath=True, nogil=True)
def numba_mul_jki(A:np.ndarray, B:np.ndarray):
    """ This is a Numba accelerated matrix-multiplication function """ 
    
    # 1. 
    # check that A and B have appropriate dimensions for multiplication
    if (np.shape(A)[1] != np.shape(B)[0]):
        return
    
    # 2. 
    # construnct a 2D numpy array for matrix C, that is filled with zerros 
    # and that has the appropriate dimensions such taht C=A*B is a valid 
    # equation and operation
    C = np.zeros((np.shape(A)[0],np.shape(B)[1]))
    
    # 3
    # Write three nested for loops, with indices, i,j,k to solve the matrix-multiplication
    # e.g.
    # for i in numba.prange(<val>):
    #   for j in range(<val>):
    #     for k in range(<val>):
    #       C[<c_row_index>,<c_col_index>] += A[<a_row_index>,<a_col_index>]*B[<b_row_index>,<b_col_index>]
    for j in numba.prange(np.shape(C)[1]):
        for k in range(np.shape(A)[1]):
            for i in range(np.shape(C)[0]):
                C[i,j] += A[i,k]*B[k,j]
    
    # 4
    # return the 2D numpy array 
    return C

@mytimer
@numba.njit(cache=True, fastmath=True, nogil=True)
def numba_mul_jik(A:np.ndarray, B:np.ndarray):
    """ This is a Numba accelerated matrix-multiplication function """ 
    
    # 1. 
    # check that A and B have appropriate dimensions for multiplication
    if (np.shape(A)[1] != np.shape(B)[0]):
        return
    
    # 2. 
    # construnct a 2D numpy array for matrix C, that is filled with zerros 
    # and that has the appropriate dimensions such taht C=A*B is a valid 
    # equation and operation
    C = np.zeros((np.shape(A)[0],np.shape(B)[1]))
    
    # 3
    # Write three nested for loops, with indices, i,j,k to solve the matrix-multiplication
    # e.g.
    # for i in numba.prange(<val>):
    #   for j in range(<val>):
    #     for k in range(<val>):
    #       C[<c_row_index>,<c_col_index>] += A[<a_row_index>,<a_col_index>]*B[<b_row_index>,<b_col_index>]
    for j in numba.prange(np.shape(C)[1]):
        for i in range(np.shape(C)[0]):
            for k in range(np.shape(A)[1]):
                C[i,j] += A[i,k]*B[k,j]
    
    # 4
    # return the 2D numpy array 
    return C

@mytimer
@numba.njit(cache=True, fastmath=True, nogil=True)
def numba_mul_kji(A:np.ndarray, B:np.ndarray):
    """ This is a Numba accelerated matrix-multiplication function """ 
    
    # 1. 
    # check that A and B have appropriate dimensions for multiplication
    if (np.shape(A)[1] != np.shape(B)[0]):
        return
    
    # 2. 
    # construnct a 2D numpy array for matrix C, that is filled with zerros 
    # and that has the appropriate dimensions such taht C=A*B is a valid 
    # equation and operation
    C = np.zeros((np.shape(A)[0],np.shape(B)[1]))
    
    # 3
    # Write three nested for loops, with indices, i,j,k to solve the matrix-multiplication
    # e.g.
    # for i in numba.prange(<val>):
    #   for j in range(<val>):
    #     for k in range(<val>):
    #       C[<c_row_index>,<c_col_index>] += A[<a_row_index>,<a_col_index>]*B[<b_row_index>,<b_col_index>]
    for k in numba.prange(np.shape(A)[1]):
        for j in range(np.shape(C)[1]):
            for i in range(np.shape(C)[0]):
                C[i,j] += A[i,k]*B[k,j]
    
    # 4
    # return the 2D numpy array 
    return C

@mytimer
@numba.njit(cache=True, fastmath=True, nogil=True)
def numba_mul_kij(A:np.ndarray, B:np.ndarray):
    """ This is a Numba accelerated matrix-multiplication function """ 
    
    # 1. 
    # check that A and B have appropriate dimensions for multiplication
    if (np.shape(A)[1] != np.shape(B)[0]):
        return
    
    # 2. 
    # construnct a 2D numpy array for matrix C, that is filled with zerros 
    # and that has the appropriate dimensions such taht C=A*B is a valid 
    # equation and operation
    C = np.zeros((np.shape(A)[0],np.shape(B)[1]))
    
    # 3
    # Write three nested for loops, with indices, i,j,k to solve the matrix-multiplication
    # e.g.
    # for i in numba.prange(<val>):
    #   for j in range(<val>):
    #     for k in range(<val>):
    #       C[<c_row_index>,<c_col_index>] += A[<a_row_index>,<a_col_index>]*B[<b_row_index>,<b_col_index>]
    for k in numba.prange(np.shape(A)[1]):
        for i in range(np.shape(C)[0]):
            for j in range(np.shape(C)[1]):
                C[i,j] += A[i,k]*B[k,j]
    
    # 4
    # return the 2D numpy array 
    return C

@mytimer
@numba.njit(cache=True, fastmath=True, nogil=True)
def numba_mul_ikj(A:np.ndarray, B:np.ndarray):
    """ This is a Numba accelerated matrix-multiplication function """ 
    
    # 1. 
    # check that A and B have appropriate dimensions for multiplication
    if (np.shape(A)[1] != np.shape(B)[0]):
        return
    
    # 2. 
    # construnct a 2D numpy array for matrix C, that is filled with zerros 
    # and that has the appropriate dimensions such taht C=A*B is a valid 
    # equation and operation
    C = np.zeros((np.shape(A)[0],np.shape(B)[1]))
    
    # 3
    # Write three nested for loops, with indices, i,j,k to solve the matrix-multiplication
    # e.g.
    # for i in numba.prange(<val>):
    #   for j in range(<val>):
    #     for k in range(<val>):
    #       C[<c_row_index>,<c_col_index>] += A[<a_row_index>,<a_col_index>]*B[<b_row_index>,<b_col_index>]
    for i in numba.prange(np.shape(C)[0]):
        for k in range(np.shape(A)[1]):
            for j in range(np.shape(C)[1]):
                C[i,j] += A[i,k]*B[k,j]
    
    # 4
    # return the 2D numpy array 
    return C

@mytimer
@numba.njit(cache=True, fastmath=True, nogil=True)
def numba_mul_ijk(A:np.ndarray, B:np.ndarray):
    """ This is a Numba accelerated matrix-multiplication function """ 
    
    # 1. 
    # check that A and B have appropriate dimensions for multiplication
    if (np.shape(A)[1] != np.shape(B)[0]):
        return
    
    # 2. 
    # construnct a 2D numpy array for matrix C, that is filled with zerros 
    # and that has the appropriate dimensions such taht C=A*B is a valid 
    # equation and operation
    C = np.zeros((np.shape(A)[0],np.shape(B)[1]))
    
    # 3
    # Write three nested for loops, with indices, i,j,k to solve the matrix-multiplication
    # e.g.
    # for i in numba.prange(<val>):
    #   for j in range(<val>):
    #     for k in range(<val>):
    #       C[<c_row_index>,<c_col_index>] += A[<a_row_index>,<a_col_index>]*B[<b_row_index>,<b_col_index>]
    for i in numba.prange(np.shape(C)[0]):
        for j in range(np.shape(C)[1]):
            for k in range(np.shape(A)[1]):
                C[i,j] += A[i,k]*B[k,j]
    
    # 4
    # return the 2D numpy array 
    return C


<br>

#### Write the test codes for Exercise 2 in the cell below.

In [16]:
# Test your code here for runtimes, accuracy, and speedups

print("Square matrices, N=128:")
A1 = np.float64(np.random.rand(128,128))
B1 = np.float64(np.random.rand(128,128))
C1np = timed_npdot(A1,B1)
C1ijk = numba_mul_ijk(A1,B1)
err1ijk = np.sum(abs(C1ijk-C1np))
print("Error {ijk} = ",err1ijk)
C1ikj = numba_mul_ikj(A1,B1)
err1ikj = np.sum(abs(C1ikj-C1np))
print("Error {ikj} = ",err1ikj)
C1kij = numba_mul_kij(A1,B1)
err1kij = np.sum(abs(C1kij-C1np))
print("Error {kij} = ",err1kij)
C1kji = numba_mul_kji(A1,B1)
err1kji = np.sum(abs(C1kji-C1np))
print("Error {kji} = ",err1kji)
C1jki = numba_mul_jki(A1,B1)
err1jki = np.sum(abs(C1jki-C1np))
print("Error {jki} = ",err1jki)
C1jik = numba_mul_jik(A1,B1)
err1jik = np.sum(abs(C1jik-C1np))
print("Error {jik} = ",err1jik)
print("Speedup of fastest ~ 15\n")

print("Square matrices, N=256:")
A2 = np.float64(np.random.rand(256,256))
B2 = np.float64(np.random.rand(256,256))
C2np = timed_npdot(A2,B2)
C2ijk = numba_mul_ijk(A2,B2)
err2ijk = np.sum(abs(C2ijk-C2np))
print("Error {ijk} = ",err2ijk)
C2ikj = numba_mul_ikj(A2,B2)
err2ikj = np.sum(abs(C2ikj-C2np))
print("Error {ikj} = ",err2ikj)
C2kij = numba_mul_kij(A2,B2)
err2kij = np.sum(abs(C2kij-C2np))
print("Error {kij} = ",err2kij)
C2kji = numba_mul_kji(A2,B2)
err2kji = np.sum(abs(C2kji-C2np))
print("Error {kji} = ",err2kji)
C2jki = numba_mul_jki(A2,B2)
err2jki = np.sum(abs(C2jki-C2np))
print("Error {jki} = ",err2jki)
C2jik = numba_mul_jik(A2,B2)
err2jik = np.sum(abs(C2jik-C2np))
print("Error {jik} = ",err2jik)
print("Speedup of fastest ~ 5\n")

print("Square matrices, N=512:")
A3 = np.float64(np.random.rand(512,512))
B3 = np.float64(np.random.rand(512,512))
C3np = timed_npdot(A3,B3)
C3ijk = numba_mul_ijk(A3,B3)
err3ijk = np.sum(abs(C3ijk-C3np))
print("Error {ijk} = ",err3ijk)
C3ikj = numba_mul_ikj(A3,B3)
err3ikj = np.sum(abs(C3ikj-C3np))
print("Error {ikj} = ",err3ikj)
C3kij = numba_mul_kij(A3,B3)
err3kij = np.sum(abs(C3kij-C3np))
print("Error {kij} = ",err3kij)
C3kji = numba_mul_kji(A3,B3)
err3kji = np.sum(abs(C3kji-C3np))
print("Error {kji} = ",err3kji)
C3jki = numba_mul_jki(A3,B3)
err3jki = np.sum(abs(C3jki-C3np))
print("Error {jki} = ",err3jki)
C3jik = numba_mul_jik(A3,B3)
err3jik = np.sum(abs(C3jik-C3np))
print("Error {jik} = ",err3jik)
print("\n\nSpeedup of fastest ~ 4\n")

C3naive = my_naive_matmul(A3,B3)
print("Speedup of fastest vs naive, N=512 ~ 600\n")

print("float32, N=512:")
A4 = np.float32(A3)
B4 = np.float32(B3)
C4ikj = numba_mul_ikj(A4,B4)
print("Speedup of float32 vs float64 ~ 2.5")

Square matrices, N=128:
Function: 'timed_npdot({})' executed in 0.000s

Function: 'numba_mul_ijk({})' executed in 0.008s

Error {ijk} =  0.0
Function: 'numba_mul_ikj({})' executed in 0.013s

Error {ikj} =  0.0
Function: 'numba_mul_kij({})' executed in 0.002s

Error {kij} =  0.0
Function: 'numba_mul_kji({})' executed in 0.007s

Error {kji} =  0.0
Function: 'numba_mul_jki({})' executed in 0.021s

Error {jki} =  0.0
Function: 'numba_mul_jik({})' executed in 0.003s

Error {jik} =  0.0
Speedup of fastest ~ 15

Square matrices, N=256:
Function: 'timed_npdot({})' executed in 0.001s

Function: 'numba_mul_ijk({})' executed in 0.076s

Error {ijk} =  1.4422809613279242e-09
Function: 'numba_mul_ikj({})' executed in 0.017s

Error {ikj} =  1.4422809613279242e-09
Function: 'numba_mul_kij({})' executed in 0.040s

Error {kij} =  1.4422809613279242e-09
Function: 'numba_mul_kji({})' executed in 0.062s

Error {kji} =  1.4422809613279242e-09
Function: 'numba_mul_jki({})' executed in 0.156s

Error {jki} =  

<br>

### Discussion for Exercise 2
1. What do you think is causing the differences in performance between the various permutations?
1. Which function is fastest (permutations or numpy.dot)? Why do you think this function is the fastest, and could there be multiple factors involved regarding the superior performance?
1. When comparing the matrix-matrix performance between the cases were all matrices had dtype=np.float64 vs.  all matrices having dtype=np.float32, which dtype was fastest? Roughtly what was the speedup when using this dtype vs the other?
1. Does Amdahl's Law play a major factor in the performance differences? Which part of the matrix-matrix multiplication was not parallelized (serial portion)? (Hint: which matricies did we reuse for each function?) Could we parallelize this part, and if so, are there caveats?
1. Do you think all codes should be parallelized? What about matrix-matrix multiplication? (This is a subjective question, but I am looking for a brief, but rational and informed arguement).
1. For those taking the class for **4 units**: Regarding the performance differences, which Law do you think is more relevant when comparing the performance differences between each of the functions in this exercise, Amdahl's Law or Gustafson's Law? Explain your reasoning. 


1. Differences in performance come from the ability to keep values stored in the cache for the entire time that value is being used. The less efficient permutations repeatedly load and delete the same values from cache, while the more efficient permutations load each value into cache as few times as possible.
2. numpy.dot is significantly faster than any of the permutations. This is because numpy.dot is highly optimized by professionals, who likely do things like divide up the problem into more than three loops that are the most efficient size for the system.
3. The float32 operations were faster, which makes sense because less data needs to be moved to perform each suboperation. The speedup was roughly ~2.5x.
4. Amdahl's law does play a major factor in performance differences. Allocating the memory for matrix C is completely serial, but could be parallelized by doing this within the second loop. Caveats include the difficulty in allocating this memory in a scalable way.
5. I think that all codes that intend to deal with large datasets should be parallelized, including matrix-matrix multiplication. The time savings for data-intensive tasks are huge. However, for less data-intensive tasks, code should be more serial to improve ease of understanding and editing.
